In [3]:
import paho.mqtt.client as mqtt
import json
import redis

# Connect to the Redis database
import redis

REDIS_HOST = 'redis-10188.c135.eu-central-1-1.ec2.cloud.redislabs.com'
REDIS_PORT = 10188
REDIS_USERNAME = 'default'
REDIS_PASSWORD = 'A7WdgwW3KBkCkq1qQxHnTt9aF4lfR6XY'

redis_client = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, username=REDIS_USERNAME, password=REDIS_PASSWORD)

is_connected = redis_client.ping()
print('Redis Connected:', is_connected)


# Define the callbacks for when the client receives a response from the server
# and when a message is published on a subscribed topic
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print('Connected to MQTT broker')
        # Subscribe to a topic when the client connects
        client.subscribe('s307798')
    else:
        print(f'Failed to connect to MQTT broker with result code {str(rc)}')

def on_message(client, userdata, msg):
    message = msg.payload.decode()
    print(f"Received message '{message}' on topic {msg.topic}")

    # Assuming the message is in JSON format
    try:
        # Parse the JSON message
        message_data = json.loads(message)
        events = message_data.get("events", [])
        mac_address = message_data.get("mac_address")
        if not redis_client.exists(f'{mac_address}:battery') or not redis_client.exists(f'{mac_address}:power'):
             redis_client.ts().create(f'{mac_address}:battery')
             redis_client.ts().create(f'{mac_address}:power')

        for event in events:
            battery_level = event.get("battery_level")
            power_plugged = event.get("power_plugged")
            timestamp = event.get("timestamp")

            if battery_level is not None and power_plugged is not None:
                redis_client.ts().add(f'{mac_address}:battery', timestamp, battery_level)
                redis_client.ts().add(f'{mac_address}:power', timestamp, power_plugged)
                print(f"Data stored in Redis TimeSeries for {mac_address} at timestamp {timestamp}")
            else:
                print("Invalid data received. Skipping storage.")
       
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
    except Exception as e:
        print(f"Error storing data in Redis: {e}")
   
# Create a new MQTT client
client = mqtt.Client()

# Set the callbacks
client.on_connect = on_connect
client.on_message = on_message

# Connect to the MQTT broker
client.connect('mqtt.eclipseprojects.io', 1883)

try:
    # Start the client loop to process network events and call the callbacks
    client.loop_forever()
except KeyboardInterrupt:
    print("not connected well run it again")
finally:
    # Perform cleanup actions if needed
    client.disconnect()
    print("MQTT client disconnected.")

Redis Connected: True
Connected to MQTT broker
Received message '{
    "mac_address": "0x3822e23d5ead",
    "events": [
        {
            "timestamp": 1706798053056,
            "battery_level": 100,
            "power_plugged": 1
        },
        {
            "timestamp": 1706798054056,
            "battery_level": 100,
            "power_plugged": 1
        },
        {
            "timestamp": 1706798055057,
            "battery_level": 100,
            "power_plugged": 1
        },
        {
            "timestamp": 1706798056057,
            "battery_level": 100,
            "power_plugged": 1
        },
        {
            "timestamp": 1706798057058,
            "battery_level": 100,
            "power_plugged": 1
        },
        {
            "timestamp": 1706798058058,
            "battery_level": 100,
            "power_plugged": 1
        },
        {
            "timestamp": 1706798059059,
            "battery_level": 100,
            "power_plugged": 1
        }

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f4c1e8e7-3e9e-4aba-a439-c0974b52f6c7' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>